## Surface data preparation for encoding/decoding models

For this example, we will use the data from Haxby et al., 2001., which are shared via OpenNeuro:

https://openneuro.org/datasets/ds000105/versions/3.0.0

The surface data were generated using fMRIPrep.

The data are formatted according to the BIDS standard: https://bids-specification.readthedocs.io/en/stable/index.html

First, import required dependencies. You can install these using `pip install -r requirements.txt` from the main repo directory.

In [29]:
%load_ext autoreload
%autoreload 2

import os
import nilearn
from nilearn import datasets
from nilearn.maskers import NiftiMasker
from nilearn.maskers import MultiNiftiMapsMasker
import h5py
import numpy as np
import nibabel as nib
import datalad.api as dl
import pandas as pd
import tqdm
import templateflow.api as tflow
from utils import (
    get_subject_common_brain_mask,
    get_group_common_mask,
    get_subject_runs,
    get_layouts,
)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### Get the data using datalad

We will use a tool called [Datalad](https://www.datalad.org/) to obtain the data from openneuro.

We will download the raw data, as well as the processed data (using [fMRIPrep](https://fmriprep.org/en/stable/).  Note that downloading these derivative data can take quite a while depending on the speed of one's connection.

In [2]:
data_dir = '/Users/poldrack/data_unsynced/ds000105'
assert os.path.exists(data_dir), 'Data directory not found: %s' % data_dir
fmriprep_dir = os.path.join(data_dir, 'derivatives', 'fmriprep')

output_dir = os.path.join(data_dir, 'derivatives', 'cleaned')
if not os.path.exists(output_dir):
    os.makedirs(output_dir)


# get the raw data
ds = dl.clone(
    path=data_dir,
    source='https://github.com/OpenNeuroDatasets/ds000105.git',
)
dl.get(dataset=data_dir, recursive=True)

get_fmriprep = False  # set to false after downloading fmriprep once

# get the preprocessed derivatives - this takes some time!
if get_fmriprep:
    dl.clone(
        path=fmriprep_dir,
        source='https://github.com/OpenNeuroDerivatives/ds000105-fmriprep.git',
    )
    dl.get(dataset=fmriprep_dir, recursive=True)

[INFO] Ensuring presence of Dataset(/Users/poldrack/data_unsynced/ds000105) to get /Users/poldrack/data_unsynced/ds000105 


### Get fsaverage5 surface

We need the surface files in order to visualize things.  We will use templateflow to get them.

In [41]:
fsavg = tflow.get('fsaverage', density='10k')
# confirm that this matches the fmriprep data
pial_files = [i for i in fsavg if 'pial' in i.name]
assert len(pial_files) == 2, 'Did not find 2 pial files in fsaverage'

pial_img = nib.load(pial_files[0])
print(pial_img.darrays[0].data.shape)


(10242, 3)


### Query the dataset using PyBIDS

Because the dataset is organized using the BIDS standard, we can use the [PyBIDS](https://bids-standard.github.io/pybids/) tool to query the dataset and obtain useful metadata.


In [3]:
# load the dataset using pybids and get runs for each subject


layout, deriv_layout = get_layouts(data_dir, fmriprep_dir)

/Users/poldrack/micromamba/envs/aineuro/lib/python3.12/site-packages/bids/layout/layout.py:516: UserWarning: Derivative indexing was requested, but no valid datasets were found in the specified locations ([PosixPath('/Users/poldrack/data_unsynced/ds000105/derivatives/fmriprep/derivatives')]). Note that all BIDS-Derivatives datasets must meet all the requirements for BIDS-Raw datasets (a common problem is to fail to include a 'dataset_description.json' file in derivatives datasets).
Example contents of 'dataset_description.json':
{"Name": "Example dataset", "BIDSVersion": "1.0.2", "GeneratedBy": [{"Name": "Example pipeline"}]}
  warnings.warn("Derivative indexing was requested, but no valid "


### Confound regression

Use the outputs from fMRIPrep to generate a denoised version of the data.



In [56]:
subject = 1
run = 1

def get_surface_files(subject, run):
    preproc_surf_files = {}
    preproc_surf_data = {}

    for hemis in ['L', 'R']:
        preproc_surf_files[hemis] = deriv_layout.get(
            subject=subject,
            run=run,
            hemi=hemis,
            space='fsaverage5',
            extension='func.gii',
            return_type='file')[0]
        preproc_surf_img = nib.load(preproc_surf_files[hemis])
        for i in range(len(preproc_surf_img.darrays)):
            if i == 0:
                preproc_surf_data[hemis] = preproc_surf_img.darrays[i].data
            else:
                preproc_surf_data[hemis] = np.vstack(
                    (preproc_surf_data[hemis], preproc_surf_img.darrays[i].data))

        print(preproc_surf_data[hemis].shape)
    return preproc_surf_files, preproc_surf_data

# make sure we have the correct fsaverage surface
preproc_surf_files, preproc_surf_data = get_surface_files(subject, run)
assert pial_img.darrays[0].data.shape[0] == preproc_surf_data['L'].shape[1]


(121, 10242)
(121, 10242)


In [59]:
def run_surface_confound_regression(layout, deriv_layout, data_dir, 
                                    overwrite=False, t_r=2.5):
    cleaned_images = {}

    subjects = [int(sub) for sub in layout.get_subjects()]
    for subject in subjects:
        cleaned_images[subject] = {}
        runs = get_subject_runs(subject, data_dir)
        print(f'Subject {subject} has {len(runs)} runs')

        t_r = None
        for run in runs:
            # get confounds which are common to run
            confound_file = deriv_layout.get(
                subject=subject,
                run=run,
                desc='confounds',
                suffix='timeseries',
                extension='tsv',
                return_type='file',
            )
            assert (
                len(confound_file) == 1
            ), f'Found {len(confound_file)} confound files for subject {subject} run {run}'
            confounds = pd.read_csv(confound_file[0], sep='\t').bfill()
            # need to include cosine with acompcor
            confound_prefixes = ['trans_', 'rot_', 'a_comp_cor_', 'cosine']
            confound_cols = [
                c
                for c in list(confounds.columns)
                if any([c.startswith(p) for p in confound_prefixes])
            ]
            confounds_selected = confounds[confound_cols]

            # get data for each hemisphere
            cleaned_images[subject][run] = {'L': None, 'R': None}
            preproc_surf_files, preproc_surf_data = get_surface_files(subject, run)
            for hemis in ['L', 'R']:
                cleaned_img_file = preproc_surf_files[hemis].replace('_bold', '_desc-cleaned_bold')
                assert cleaned_img_file != preproc_surf_files[hemis]
                if os.path.exists(cleaned_img_file) and not overwrite:
                    # print(f"Using existing cleaned file for subject {subject} run {run}")
                    cleaned_img = nib.load(cleaned_img_file)
                    cleaned_images[subject][run][hemis] = (cleaned_img_file, cleaned_img)
                    continue
                assert (
                    len(preproc_surf_files) == 2
                ), f'Found {len(preproc_file)} preproc files for subject {subject} run {run}'
                cleaned_data = nilearn.signal.clean(
                    preproc_surf_data[hemis],
                    confounds=confounds_selected,
                    t_r=t_r,
                    standardize='zscore_sample',)
                cleaned_img = nib.gifti.GiftiImage()
                preproc_surf_img = nib.load(preproc_surf_files[hemis])
                for i in range(cleaned_data.shape[0]):
                    cleaned_img.add_gifti_data_array(
                        nib.gifti.GiftiDataArray(
                            cleaned_data[i, :],
                            intent=preproc_surf_img.darrays[i].intent,
                            datatype=preproc_surf_img.darrays[i].datatype,
                            meta=preproc_surf_img.darrays[i].meta,
                        )
                    )
                cleaned_img.to_filename(os.path.join(cleaned_img_file))
                cleaned_images[subject][run] = (cleaned_img_file, cleaned_img)
    return cleaned_images, t_r


# refresh the layouts to detect the new res-3 files
layout, deriv_layout = get_layouts(data_dir, fmriprep_dir)

cleaned_images, t_r = run_surface_confound_regression(
    layout, deriv_layout, data_dir, overwrite=True
)


/Users/poldrack/micromamba/envs/aineuro/lib/python3.12/site-packages/bids/layout/layout.py:516: UserWarning: Derivative indexing was requested, but no valid datasets were found in the specified locations ([PosixPath('/Users/poldrack/data_unsynced/ds000105/derivatives/fmriprep/derivatives')]). Note that all BIDS-Derivatives datasets must meet all the requirements for BIDS-Raw datasets (a common problem is to fail to include a 'dataset_description.json' file in derivatives datasets).
Example contents of 'dataset_description.json':
{"Name": "Example dataset", "BIDSVersion": "1.0.2", "GeneratedBy": [{"Name": "Example pipeline"}]}
  warnings.warn("Derivative indexing was requested, but no valid "


Subject 1 has 12 runs
(121, 10242)
(121, 10242)


/var/folders/r2/f85nyfr1785fj4257wkdj7480000gn/T/ipykernel_77676/893423826.py:49: DeprecationWarning: The default strategy for standardize is currently 'zscore' which incorrectly uses population std to calculate sample zscores. The new strategy 'zscore_sample' corrects this behavior by using the sample std. In release 0.13, the default strategy will be replaced by the new strategy and the 'zscore' option will be removed. Please use 'zscore_sample' instead.
  cleaned_data = nilearn.signal.clean(


(121, 10242)
(121, 10242)


/var/folders/r2/f85nyfr1785fj4257wkdj7480000gn/T/ipykernel_77676/893423826.py:49: DeprecationWarning: The default strategy for standardize is currently 'zscore' which incorrectly uses population std to calculate sample zscores. The new strategy 'zscore_sample' corrects this behavior by using the sample std. In release 0.13, the default strategy will be replaced by the new strategy and the 'zscore' option will be removed. Please use 'zscore_sample' instead.
  cleaned_data = nilearn.signal.clean(


...

/var/folders/r2/f85nyfr1785fj4257wkdj7480000gn/T/ipykernel_77676/893423826.py:49: DeprecationWarning: The default strategy for standardize is currently 'zscore' which incorrectly uses population std to calculate sample zscores. The new strategy 'zscore_sample' corrects this behavior by using the sample std. In release 0.13, the default strategy will be replaced by the new strategy and the 'zscore' option will be removed. Please use 'zscore_sample' instead.
  cleaned_data = nilearn.signal.clean(


(121, 10242)
(121, 10242)


/var/folders/r2/f85nyfr1785fj4257wkdj7480000gn/T/ipykernel_77676/893423826.py:49: DeprecationWarning: The default strategy for standardize is currently 'zscore' which incorrectly uses population std to calculate sample zscores. The new strategy 'zscore_sample' corrects this behavior by using the sample std. In release 0.13, the default strategy will be replaced by the new strategy and the 'zscore' option will be removed. Please use 'zscore_sample' instead.
  cleaned_data = nilearn.signal.clean(


(121, 10242)
(121, 10242)


/var/folders/r2/f85nyfr1785fj4257wkdj7480000gn/T/ipykernel_77676/893423826.py:49: DeprecationWarning: The default strategy for standardize is currently 'zscore' which incorrectly uses population std to calculate sample zscores. The new strategy 'zscore_sample' corrects this behavior by using the sample std. In release 0.13, the default strategy will be replaced by the new strategy and the 'zscore' option will be removed. Please use 'zscore_sample' instead.
  cleaned_data = nilearn.signal.clean(


In [60]:
cleaned_images

{1: {1: ('/Users/poldrack/data_unsynced/ds000105/derivatives/fmriprep/sub-1/func/sub-1_task-objectviewing_run-1_hemi-R_space-fsaverage5_desc-cleaned_bold.func.gii',
   <nibabel.gifti.gifti.GiftiImage at 0x336c14e60>),
  2: ('/Users/poldrack/data_unsynced/ds000105/derivatives/fmriprep/sub-1/func/sub-1_task-objectviewing_run-2_hemi-R_space-fsaverage5_desc-cleaned_bold.func.gii',
   <nibabel.gifti.gifti.GiftiImage at 0x335caa8a0>),
  3: ('/Users/poldrack/data_unsynced/ds000105/derivatives/fmriprep/sub-1/func/sub-1_task-objectviewing_run-3_hemi-R_space-fsaverage5_desc-cleaned_bold.func.gii',
   <nibabel.gifti.gifti.GiftiImage at 0x335caa900>),
  4: ('/Users/poldrack/data_unsynced/ds000105/derivatives/fmriprep/sub-1/func/sub-1_task-objectviewing_run-4_hemi-R_space-fsaverage5_desc-cleaned_bold.func.gii',
   <nibabel.gifti.gifti.GiftiImage at 0x334b389b0>),
  5: ('/Users/poldrack/data_unsynced/ds000105/derivatives/fmriprep/sub-1/func/sub-1_task-objectviewing_run-5_hemi-R_space-fsaverage5_desc

### select task block timepoints

drop the first two TRs from each task block, and generate task labels for each timepoint

In [61]:
# find the condition label for timepoints that are beyond the intial 4 seconds
def get_cond_info(
    layout, deriv_layout, t_r, cleaned_images, blocklen=20, n_trials_to_skip=2
):
    cond_info = {}

    subjects = [int(sub) for sub in layout.get_subjects()]
    for subject in subjects:
        runs = get_subject_runs(subject, data_dir)
        cond_info[subject] = {}
        for run in runs:
            events_file = layout.get(
                subject=subject,
                run=run,
                datatype='func',
                extension='tsv',
                return_type='file',
            )[0]
            events = pd.read_csv(events_file, sep='\t')
            n_timepoints = cleaned_images[subject][run][1].shape[-1]
            timepoints = np.arange(0, n_timepoints * t_r, t_r)

            # find task onsets in the events file
            # skip 2 trials i.e. 4 seconds
            blocklen = (
                20  # block length in seconds after removing first 4 seconds
            )

            conditions = events.trial_type.unique().tolist()
            conditions.sort()
            onsets = {}
            for condition in conditions:
                match_df = events[events.trial_type == condition]
                onsets[condition] = match_df.onset.tolist()[n_trials_to_skip]
            cond_df = pd.DataFrame(
                {'timepoint': timepoints, 'condition': None}
            )
            for idx in cond_df.index:
                for condition in conditions:
                    if cond_df.loc[idx, 'timepoint'] >= onsets[
                        condition
                    ] and cond_df.loc[idx, 'timepoint'] < (
                        onsets[condition] + blocklen
                    ):
                        cond_df.loc[idx, 'condition'] = condition
            for cond in cond_df.condition.unique():
                if cond is None:
                    continue
                assert len(cond_df[cond_df.condition == cond]) == 8
            cond_info[subject][run] = cond_df
    return cond_info


cond_info = get_cond_info(layout, deriv_layout, t_r, cleaned_images)

AttributeError: 'GiftiImage' object has no attribute 'shape'

In [ ]:


def get_task_images(cond_info, cleaned_images):
    task_images = {}
    task_info = {}
    for subject, runs in cond_info.items():
        task_images[subject] = {}
        task_info[subject] = {}
        for run, cond_df in runs.items():
            good_trials = cond_df.dropna()
            assert (
                len(good_trials) == 64
            ), f'Found {len(good_trials)} good trials for subject {subject} run {run}'
            task_img_file = cleaned_images[subject][run][0].replace(
                'cleaned', 'task'
            )
            assert task_img_file != cleaned_images[subject][run][0]
            good_trials.to_csv(
                task_img_file.replace('_bold.nii.gz', '_events.tsv'),
                sep='\t',
                index=False,
            )
            task_info[subject][run] = good_trials
            if not os.path.exists(task_img_file):
                cleaned_img = cleaned_images[subject][run][1]
                task_data = cleaned_img.get_fdata()[
                    :, :, :, list(good_trials.index)
                ]
                task_img = nib.Nifti1Image(task_data, cleaned_img.affine)
                task_img.to_filename(task_img_file)
            else:
                task_img = nib.load(task_img_file)
            task_images[subject][run] = task_img
    return task_images, task_info


task_images, task_info = get_task_images(cond_info, cleaned_images)

### save to HDF5 file

In [ ]:
subjects = [int(sub) for sub in layout.get_subjects()]


difumo = datasets.fetch_atlas_difumo(
    dimension=1024, resolution_mm=3, legacy_format=False
)
difumo_masker = MultiNiftiMapsMasker(
    maps_img=difumo.maps,
    n_jobs=12,
)

vtmaskdir = os.path.join(data_dir, 'derivatives', 'vtmasks')
with h5py.File(os.path.join(output_dir, 'haxby_data_cleaned.h5'), 'w') as hf:

    for subject in subjects:
        print(f'Processing subject {subject}')
        g1 = hf.create_group(f'sub-{subject}')

        runs = get_subject_runs(subject, data_dir)
        sub_mask = get_subject_common_brain_mask(subject, data_dir)
        masker = NiftiMasker(mask_img=sub_mask)
        vt_mask = nib.load(
            os.path.join(
                vtmaskdir,
                f'sub-{subject}_mask4vt_space-MNI152NLin2009cAsym_res-3.nii.gz',
            )
        )

        vtmasker = NiftiMasker(mask_img=vt_mask)
        for run in runs:
            g2 = g1.create_group(f'run-{run}')

            vt_data = vtmasker.fit_transform(task_images[subject][run])
            assert vt_data.shape[0] == task_images[subject][run].shape[-1]
            assert vt_data.shape[1] == np.sum(vt_mask.get_fdata())
            g2.create_dataset('vtmaskdata', data=vt_data)

            # get the whole brain data
            braindata = masker.fit_transform(task_images[subject][run])
            assert braindata.shape[0] == task_images[subject][run].shape[-1]
            assert braindata.shape[1] == np.sum(sub_mask.get_fdata())
            g2.create_dataset('braindata', data=braindata)

            g2.create_dataset(
                'conditions',
                data=[
                    c.encode('utf-8')
                    for c in task_info[subject][run].condition.tolist()
                ],
            )

            # get the difumo data
            difumo_data = difumo_masker.fit_transform(
                task_images[subject][run]
            )
            assert difumo_data.shape[0] == task_images[subject][run].shape[-1]
            g2.create_dataset('difumodata', data=difumo_data)

### Average within blocks



In [ ]:
with h5py.File(os.path.join(output_dir, 'haxby_data_cleaned.h5'), 'a') as hf:

    subjects = [int(sub) for sub in layout.get_subjects()]
    for subject in subjects:
        runs = get_subject_runs(subject, data_dir)
        for run in runs:
            conditions = [
                i.decode('utf-8')
                for i in hf[f'sub-{subject}/run-{run}']['conditions'][:]
            ]

            vt_df = pd.DataFrame(
                hf[f'sub-{subject}/run-{run}']['vtmaskdata'][:]
            )
            condition_mean_df = vt_df.groupby(conditions).mean().sort_index()
            conditions_collapsed = condition_mean_df.index.values
            hf[f'sub-{subject}/run-{run}'].create_dataset(
                'mean_vtmaskdata', data=condition_mean_df
            )
            hf[f'sub-{subject}/run-{run}'].create_dataset(
                'mean_conditions', data=condition_mean_df.index.values
            )

            # same for whole brain
            brain_df = pd.DataFrame(
                hf[f'sub-{subject}/run-{run}']['braindata'][:]
            )
            condition_mean_df = (
                brain_df.groupby(conditions).mean().sort_index()
            )
            # make sure conditions are the same between the two
            assert np.all(
                condition_mean_df.index.values == conditions_collapsed
            )
            hf[f'sub-{subject}/run-{run}'].create_dataset(
                'mean_braindata', data=condition_mean_df
            )

            # same for difumo
            difumo_df = pd.DataFrame(
                hf[f'sub-{subject}/run-{run}']['difumodata'][:]
            )
            condition_mean_df = (
                difumo_df.groupby(conditions).mean().sort_index()
            )
            # make sure conditions are the same between the two
            assert np.all(
                condition_mean_df.index.values == conditions_collapsed
            )
            hf[f'sub-{subject}/run-{run}'].create_dataset(
                'mean_difumodata', data=condition_mean_df
            )

In [ ]:
condition_mean_df.sort_index()

In [ ]:

with h5py.File(os.path.join(output_dir, 'haxby_data_cleaned.h5'), 'r') as hf:

    # utils.list_all_datasets(hf)
    print(hf[f'sub-{subject}/run-{run}']['braindata'].shape)
    print(hf[f'sub-{subject}/run-{run}']['vtmaskdata'].shape)
    print(hf[f'sub-{subject}/run-{run}']['difumodata'].shape)

    print(hf[f'sub-{subject}/run-{run}']['conditions'].shape)
    print(hf[f'sub-{subject}/run-{run}']['mean_conditions'].shape)